In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
# Import Area Under the Receiver Operating Characteristic Curve metric to evaluate results
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline

In [2]:
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelBinarizer

In [5]:
#Leo el csv events
events_df = pd.read_csv('/home/miki_mustard/Desktop/Facultad/Datos/TP2/7506_ml-master/9_11_mas_supports.csv', low_memory=False)

In [6]:
events_df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,support_ind,year,month_sin,month_cos,day_sin,day_cos,weekday_sin,weekday_cos,hour_sin,hour_cos
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,0.956244,2018,0.5,-0.866025,-0.485302,-0.874347,0.0,1.0,0.0,1.0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,0.956244,2018,0.5,-0.866025,-0.485302,-0.874347,0.0,1.0,0.0,1.0
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,0.956244,2018,0.5,-0.866025,-0.485302,-0.874347,0.0,1.0,0.0,1.0
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,0.956244,2018,0.5,-0.866025,-0.485302,-0.874347,0.0,1.0,0.0,1.0
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,0.956244,2018,0.5,-0.866025,-0.485302,-0.874347,0.0,1.0,0.0,1.0


In [98]:
#event = pd.read_csv('/home/miki_mustard/pred_con_feats.csv', low_memory=False)

In [8]:
#Leo el csv labels
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [99]:
#label = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [67]:
#diccionario_person = generar_dict(labels_df.person)
#person_numero = generar_columna(labels_df['person'], diccionario_person)

In [68]:
#labels_df['person_numero_lb'] = person_numero

In [5]:
#labels_df.head()

In [6]:
#labels = labels_df[ ['label', 'person_numero_lb']]

In [7]:
#labels.head()

In [8]:
events_df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,q2,q3,q4,q5,q6,q7,q8,q9,q10,support_p_todos_sus_ev
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,False,False,False,False,False,False,False,False,True,0.218522
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,False,False,False,False,False,False,False,False,True,0.260913
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64GB,Prateado,NaN,...,False,False,False,False,False,False,False,False,True,0.008396
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128GB,Vermelho,NaN,...,False,False,False,False,False,False,False,False,True,0.138376
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32GB,Branco,NaN,...,False,False,False,False,False,False,False,False,True,0.038811


In [9]:
#Creo el dataframe con las personas que tienen labels
# armo df con registros completos clasificados
train_df = events_df.merge(labels_df.set_index("person"), on='person', how='right')

In [10]:
train_df.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'mes', 'dia', 'q1', 'q2',
       'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10',
       'support_p_todos_sus_ev', 'support_con_peso', 'support_ind', 'year',
       'month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin',
       'weekday_cos', 'hour_sin', 'hour_cos', 'label'],
      dtype='object')

In [11]:
train_df.shape

(1171886, 48)

In [12]:
# armo df con registros a predecir unicamente
to_predict = events_df[~events_df.person.isin(labels_df.person)]

In [13]:
test_size = 0.33
# define a seed, so same experiments output same results every time
seed = 12
# realizo train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df.loc[:, train_df.columns != 'label'], 
                                                    train_df.label, 
                                                    test_size=test_size, 
                                                    random_state=seed)

In [136]:
lista_cols = [1,2,3,5,6,8,9,10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]

Ahora hago el preprocesamiento para que todos los features sean numeros

In [36]:
# some date processing
def date_proc(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['year'] = df['timestamp'].dt.year
    df['month'] = df['timestamp'].dt.month
    df['day'] = df['timestamp'].dt.day
    df['weekday'] = df['timestamp'].dt.day_name()
    df['hour'] = df['timestamp'].dt.hour
    df['year_month_day'] = df['timestamp'].map(lambda x: str(x.year)+"/"+str(x.month)+"/"+str(x.day))
    df['year_month_day'] = pd.to_datetime(df['year_month_day'])
    


In [44]:
date_proc(train_df)

In [150]:
date_proc(to_predict)

/home/miki_mustard/envdatos/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/miki_mustard/envdatos/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/miki_mustard/envdatos/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

KeyboardInterrupt: 

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/miki_mustard/envdatos/etc/spark-2.3.0-bin-hadoop2.7/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1055, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.6/socket.py", line 586, in readinto
    return self._sock.recv_into(b)
  File "/home/miki_mustard/envdatos/etc/spark-2.3.0-bin-hadoop2.7/python/pyspark/context.py", line 233, in signal_handler
    self.cancelAllJobs()
  File "/home/miki_mustard/envdatos/etc/spark-2.3.0-bin-hadoop2.7/python/pyspark/context.py", line 971, in cancelAllJobs
    self._jsc.sc().cancelAllJobs()
  File "/home/miki_mustard/envdatos/etc/spark-2.3.0-bin-hadoop2.7/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 1160, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/home/miki_mustard/envdatos/etc/spark-2.3.0-bin-hadoop2.7/python/pyspark/sql/utils.py", line 63, in deco
    

In [45]:
train_df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,q9,q10,support_p_todos_sus_ev,label,year,month,day,weekday,hour,year_month_day
0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,False,True,0.260913,0,2018,5,18,Friday,0,2018-05-18
1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.0,iPhone 5s,Muito Bom,64GB,Prateado,NaN,...,False,True,0.260913,0,2018,5,18,Friday,0,2018-05-18
2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,5907.0,iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,...,False,True,0.260913,0,2018,5,18,Friday,0,2018-05-18
3,2018-05-18 00:14:55,viewed product,ad93850f,NaN,6023.0,iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,NaN,...,False,True,0.260913,0,2018,5,18,Friday,0,2018-05-18
4,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,NaN,NaN,NaN,NaN,NaN,NaN,...,False,True,0.260913,0,2018,5,18,Friday,0,2018-05-18


In [46]:
weekday_le = preprocessing.LabelEncoder()
weekday_le.fit(train_df.weekday)

events_df.weekday = weekday_le.transform(train_df.weekday)

In [47]:
def month_to_cyclic(df):
    df['month_sin'] = df['month'].apply(lambda x: np.sin(2*np.pi*x/12))
    df['month_cos'] = df['month'].apply(lambda x: np.cos(2*np.pi*x/12))
    df.drop('month', axis=1, inplace=True)
    
def day_to_cyclic(df):
    df['day_sin'] = df['day'].apply(lambda x: np.sin(2*np.pi*x/31))
    df['day_cos'] = df['day'].apply(lambda x: np.cos(2*np.pi*x/31))
    df.drop('day', axis=1, inplace=True)

def weekday_to_cyclic(df):
    df['weekday_sin'] = df['weekday'].apply(lambda x: np.sin(2*np.pi*x/7))
    df['weekday_cos'] = df['weekday'].apply(lambda x: np.cos(2*np.pi*x/7))
    df.drop('weekday', axis=1, inplace=True)

def hour_to_cyclic(df):
    df['hour_sin'] = df['hour'].apply(lambda x: np.sin(2*np.pi*x/24))
    df['hour_cos'] = df['hour'].apply(lambda x: np.cos(2*np.pi*x/24))
    df.drop('hour', axis=1, inplace=True)

In [48]:
month_to_cyclic(train_df)
day_to_cyclic(train_df)
weekday_to_cyclic(train_df)
hour_to_cyclic(train_df)

TypeError: can't multiply sequence by non-int of type 'float'

In [49]:
train_df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,support_p_todos_sus_ev,label,year,weekday,hour,year_month_day,month_sin,month_cos,day_sin,day_cos
0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,0.260913,0,2018,Friday,0,2018-05-18,0.5,-0.866025,-0.485302,-0.874347
1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.0,iPhone 5s,Muito Bom,64GB,Prateado,NaN,...,0.260913,0,2018,Friday,0,2018-05-18,0.5,-0.866025,-0.485302,-0.874347
2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,5907.0,iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,...,0.260913,0,2018,Friday,0,2018-05-18,0.5,-0.866025,-0.485302,-0.874347
3,2018-05-18 00:14:55,viewed product,ad93850f,NaN,6023.0,iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,NaN,...,0.260913,0,2018,Friday,0,2018-05-18,0.5,-0.866025,-0.485302,-0.874347
4,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,NaN,NaN,NaN,NaN,NaN,NaN,...,0.260913,0,2018,Friday,0,2018-05-18,0.5,-0.866025,-0.485302,-0.874347


In [144]:
def pasar_a_num(df, lista_cols):
    df_result = pd.DataFrame()
    for i in range (1,2,3,5,6,8,9,10,11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22):
        palabra = lista_cols[i]
        diccionario = generar_dict(df.palabra )
        numero = generar_columna(df[palabra], diccionario)
        df_result[palabra] = numero 
    return df_result   
    

In [62]:
#Genera un dict donde para cada valor que toma la columna le asigna un numero
def generar_dict(columna):
    diccionario = {}
    unique = columna.unique()
    for i in range (0, len(unique)):
        diccionario[unique[i]] = i
    return diccionario    

In [63]:
#Arma la columna pasando la variable a su valor numerico
def generar_columna(columna_original, diccionario):
    columna_nueva = list()
    for i in range (0, len(columna_original)):
        columna_nueva.append( diccionario[columna_original[i]] )
    return columna_nueva              

In [145]:
train = pasar_a_num(train_df, lista_cols)

TypeError: range expected at most 3 arguments, got 20

Continuo con el preprocesamiento 

In [64]:
train = pd.DataFrame()

## Browser version

In [65]:
diccionario_browser = generar_dict(train_df.browser_version)
browser_numero = generar_columna(train_df['browser_version'], diccionario_browser)

In [19]:
len(browser_numero)

2341681

In [66]:
events_df.shape

(2341681, 36)

In [67]:
train['browser_version_n'] = browser_numero 

## Operating_system_version

In [68]:
diccionario_os = generar_dict(train_df.operating_system_version)
os_numero = generar_columna(train_df['operating_system_version'], diccionario_os)

In [24]:
len(os_numero) == events_df.shape[0]

True

In [69]:
train['operating_system_version_n'] = os_numero 

## screen_resolution

In [70]:
def get_screen_width(x):
    if x != "":
        return int(x.split("x")[0])
    else:
        return 0
    
def get_screen_height(x):
    if x != "":
        return int(x.split("x")[1])
    else:
        return 0

def process_screen_res(df):
    df['screen_resolution'].fillna("", inplace=True)
    df['screen_width'] = df['screen_resolution'].apply(lambda x: get_screen_width(x))
    df['screen_height'] = df['screen_resolution'].apply(lambda x: get_screen_height(x))
    df.drop('screen_resolution', axis=1, inplace=True)

In [71]:
process_screen_res(train_df)

## device_type

In [72]:
events_df.device_type.unique()

array([nan, 'Smartphone', 'Computer', 'Tablet', 'Unknown'], dtype=object)

In [73]:
diccionario_device = generar_dict(train_df.device_type)
device_numero = generar_columna(train_df['device_type'], diccionario_device)

In [125]:
len(device_numero) == events_df.shape[0]

True

In [74]:
train['device_type_n'] = os_numero 

## country

In [27]:
print(events_df.country.unique())
print("\n\tlen: " + str(len(events_df.country.unique())))

[nan 'Brazil' 'Unknown' 'Netherlands' 'United States' 'Italy' 'Argentina'
 'Dominican Republic' 'France' 'Algeria' 'Mexico' 'Canada'
 'United Kingdom' 'Paraguay' 'Peru' 'South Africa' 'Jamaica' 'Georgia'
 'Belgium' 'Spain' 'Israel' 'Bangladesh' 'Cape Verde' 'Portugal'
 'Guadeloupe' 'Mozambique' 'Germany' 'Vietnam' 'Uruguay' 'Ireland' 'India'
 'Singapore' 'Bolivia' 'Japan' 'Costa Rica' 'Romania' 'Sweden'
 'Slovak Republic' 'Colombia' 'Morocco' 'Burundi' 'Austria' 'Russia'
 'Ukraine' 'Angola' 'Bulgaria' 'Pakistan' 'Republic of Korea'
 'São Tomé and Príncipe' 'Cameroon' 'Philippines' 'Guinea-Bissau']

	len: 52


In [75]:
diccionario_country = generar_dict(train_df.country)
country_numero = generar_columna(train_df['country'], diccionario_country)

In [76]:
train['country_n'] = country_numero 

## region

In [31]:
print(events_df.region.unique())
print("\n\tlen: " + str(len(events_df.region.unique())))

[nan 'Parana' 'Unknown' 'Sao Paulo' 'Santa Catarina' 'Piaui'
 'Rio de Janeiro' 'Goias' 'Maranhao' 'Minas Gerais' 'Espirito Santo'
 'Bahia' 'Pernambuco' 'Rio Grande do Sul' 'Para' 'Paraíba' 'Ceara'
 'Federal District' 'Tocantins' 'Rondonia' 'Amazonas'
 'Rio Grande do Norte' 'Mato Grosso' 'Mato Grosso do Sul' 'Alagoas'
 'Sergipe' 'Amapa' 'North Holland' 'Turin' 'California' 'Illinois'
 'New Jersey' 'Acre' 'New York' 'Texas' 'Nacional' 'Florida' 'Mexico City'
 'Roraima' 'Arizona' 'Ontario' 'North' 'Massachusetts' 'Western Cape'
 'Kingston' 'Brussels Capital' 'Paris' 'Navarre' 'Quebec'
 'British Columbia' 'Buenos Aires F.D.' 'Dhaka' 'Praia' 'Ohio' 'Iowa'
 'Lisbon' 'Faro' 'Sofala Province' 'Delaware' 'Aveiro' 'Michigan'
 'Virginia' 'Ho Chi Minh City' 'Departamento de Montevideo' 'Colorado'
 'Georgia' 'Connecticut' 'Wisconsin' 'Rhône' 'Seine-Saint-Denis'
 'Tennessee' 'Entre Rios' 'Neuquen' 'County Cork' 'Buenos Aires'
 'Washington' 'Flemish Brabant Province' 'Hesse' 'Tinh Thua Thien-Hue'
 'K

In [77]:
diccionario_region = generar_dict(train_df.region)
region_numero = generar_columna(train_df['region'], diccionario_region)

In [133]:
len(region_numero) == events_df.shape[0]

True

In [78]:
train['region_n'] = region_numero 

## city

In [36]:
print(events_df.city.unique())
print("\n\tlen: " + str(len(events_df.city.unique())))

[nan 'Nova Esperanca' 'Unknown' ... 'Braco do Norte' 'Querencia do Norte'
 'Varjota']

	len: 2207


In [79]:
diccionario_city = generar_dict(train_df.city)
city_numero = generar_columna(train_df['city'], diccionario_city)

In [137]:
len(city_numero) == events_df.shape[0]

True

In [80]:
train['city_n'] = city_numero 

## new_vs_returning

In [39]:
print(events_df.new_vs_returning.unique())
print("\n\tlen: " + str(len(events_df.new_vs_returning.unique())))

[nan 'Returning' 'New']

	len: 3


In [81]:
diccionario_nvr = generar_dict(train_df.new_vs_returning)
nvr_numero = generar_columna(train_df['new_vs_returning'], diccionario_nvr)

In [141]:
len(nvr_numero) == events_df.shape[0]

True

In [82]:
train['new_vs_returning_n'] = nvr_numero 

## channel

In [41]:
print(events_df.channel.unique())
print("\n\tlen: " + str(len(events_df.channel.unique())))

[nan 'Paid' 'Organic' 'Direct' 'Referral' 'Social' 'Email' 'Unknown']

	len: 8


In [83]:
diccionario_chan = generar_dict(train_df.channel)
chan_numero = generar_columna(train_df['channel'], diccionario_chan)

In [144]:
len(chan_numero) == events_df.shape[0]

True

In [84]:
train['channel_n'] = city_numero 

## search_engine

In [43]:
print(events_df.search_engine.unique())
print("\n\tlen: " + str(len(events_df.search_engine.unique())))

[nan 'Google' 'Yahoo' 'Bing' 'Ask']

	len: 5


In [85]:
diccionario_se = generar_dict(train_df.search_engine)
se_numero = generar_columna(train_df['search_engine'], diccionario_se)

In [147]:
len(se_numero) == events_df.shape[0]

True

In [86]:
train['search_engine_n'] = se_numero 

## campaign_source

In [45]:
print(events_df.campaign_source.unique())
print("\n\tlen: " + str(len(events_df.campaign_source.unique())))

[nan 'google' 'afilio' 'manifest' 'rtbhouse' 'criteo' 'rakuten' 'voxus'
 'zanox' 'emblue' 'datacrush' 'buscape' 'FacebookAds' 'blog' 'bing'
 'indexa' 'Facebook' 'yotpo' 'afiliado' 'mercadopago' 'FacebookSocial'
 'MARKETING SOCIAL' 'onsite' 'gizmodo']

	len: 24


In [87]:
diccionario_cs = generar_dict(train_df.campaign_source)
cs_numero = generar_columna(train_df['campaign_source'], diccionario_cs)

In [150]:
len(cs_numero) == events_df.shape[0]

True

In [88]:
train['campaign_source_n'] = cs_numero 

## staticpage*

In [47]:
print(events_df.staticpage.unique())
print("\n\tlen: " + str(len(events_df.staticpage.unique())))

[nan 'Conditions' 'AboutUs' 'how-to-sell' 'trust-trocafone' 'how-to-buy'
 'CustomerService' 'club-trocafone' 'Quiosks' 'FaqEcommerce'
 'TermsAndConditionsReturnEcommerce' 'TermsAndConditionsEcommerce'
 'galaxy-s8' 'PrivacyEcommerce' 'black_friday']

	len: 15


In [89]:
diccionario_sp = generar_dict(train_df.staticpage)
sp_numero = generar_columna(train_df['staticpage'], diccionario_sp)

In [153]:
len(sp_numero) == events_df.shape[0]

True

In [90]:
train['staticpage_n'] = sp_numero 

## search_term

In [49]:
print(events_df.search_term.unique())
print("\n\tlen: " + str(len(events_df.search_term.unique())))

[nan 'iPhone se' '6splus' ... 'moro g5 plus' 'S8 edege' 'motp zc']

	len: 10965


In [91]:
diccionario_st = generar_dict(train_df.search_term)
st_numero = generar_columna(train_df['search_term'], diccionario_st)

In [156]:
len(st_numero) == events_df.shape[0]

True

In [92]:
train['search_term_n'] = st_numero 

## skus

In [51]:
print(events_df.skus.unique())
print("\n\tlen: " + str(len(events_df.skus.unique())))

[nan
 '10240,9987,10322,10085,9944,9931,13404,10154,10113,10294,10337,10126'
 '3371,6357,6371,3191,2718,6001,2777,6248,2725,8541,2893,6791' ...
 '12605,788,2774,2814,8275,5281,12478,2783,12395,12436,2780,2784'
 '8582,8569,9202,7965,7825,8009,9175,3109,9188,3121,8931,10450'
 '10322,9902,9972,10084,9944,9930,10100,10044,11906,9959']

	len: 52268


In [93]:
diccionario_skus = generar_dict(train_df.skus)
skus_numero = generar_columna(train_df['skus'], diccionario_skus)

In [159]:
len(skus_numero) == events_df.shape[0]

True

In [94]:
train['skus_n'] = skus_numero 

## color

In [53]:
print(events_df.color.unique())
print("\n\tlen: " + str(len(events_df.color.unique())))

['Dourado' 'Cinza espacial' 'Prateado' 'Vermelho' 'Branco' nan 'Platinum'
 'Ouro' 'Preto Vermelho' 'Preto' 'Ouro Rosa' 'Ametista' 'Rosa'
 'Couro Vintage' 'Prata' 'Bambu' 'Preto Matte' 'Preto Verde'
 'Branco Vermelho' 'Azul' 'Rouge' 'Verde' 'Rose' 'Preto Brilhante'
 'Branco Pink' 'Preto Azul' 'Titânio' 'Cinza' 'Branco Framboesa'
 'Branco Azul Navy' 'Indigo' 'Amarelo' 'Iuna' 'Azul Safira' 'Black Piano'
 'Verde Água' 'Branco Azul' 'Cabernet' 'Cobre' 'Olympic Edition'
 'Branco Cabernet' 'Preto Azul Navy' 'Azul Topázio' 'Couro Navy'
 'Azul Escuro' 'Branco Bambu' 'Preto Pink' 'Preto Tabaco' 'Branco Dourado'
 'Verde Petroleo' 'Turquesa' 'Preto Bambu' 'Couro Marrom' 'Couro Vinho'
 'Roxo' 'Cromo' 'Coral' 'Preto Cabernet' 'Branco Verde' 'Framboesa'
 'Preto Branco' 'Silver' 'Ônix' 'Preto Asfalto']

	len: 64


In [95]:
diccionario_color = generar_dict(train_df.color)
color_numero = generar_columna(train_df['color'], diccionario_color)

In [162]:
len(color_numero) == events_df.shape[0]

True

In [96]:
train['color_n'] = color_numero 

## storage

In [97]:
def process_storage_string(x):
    if pd.isna(x):
        return 0
    s = x.split("GB")
    if len(s) == 2:
        # case data in GB
        return int(s[0])
    else:
        # case data in MB
        return int(x.split("MB")[0])/1024

def storage_process(df):
    df.storage = df.storage.apply(lambda x: process_storage_string(x))

In [98]:
storage_process(events_df)

In [52]:
events_df.head()

,timestamp,event,person,url,sku,model,condition,storage,screen_resolution,mes,...,region_numero,city_numero,nvr_numero,chan_numero,se_numero,cs_numero,sp_numero,st_numero,skus_numero,color_numero
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32.0,NaN,5,...,0,0,0,0,0,0,0,0,0,0
1,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32.0,NaN,5,...,0,0,0,0,0,0,0,0,0,1
2,2018-05-18 00:11:16,viewed product,0297fc1e,NaN,6888.0,iPhone 6S,Muito Bom,64.0,NaN,5,...,0,0,0,0,0,0,0,0,0,2
3,2018-05-18 00:11:14,viewed product,2d681dd8,NaN,11890.0,iPhone 7,Bom,128.0,NaN,5,...,0,0,0,0,0,0,0,0,0,3
4,2018-05-18 00:11:09,viewed product,cccea85e,NaN,7517.0,LG G4 H818P,Excelente,32.0,NaN,5,...,0,0,0,0,0,0,0,0,0,4


## condition

In [57]:
print(events_df.condition.unique())
print("\n\tlen: " + str(len(events_df.condition.unique())))

['Excelente' 'Muito Bom' 'Bom' nan 'Bom - Sem Touch ID' 'Novo']

	len: 6


In [99]:
diccionario_cond = generar_dict(train_df.condition)
cond_numero = generar_columna(train_df['condition'], diccionario_cond)

In [169]:
len(cond_numero) == events_df.shape[0]

True

In [100]:
train['condition_n'] = cond_numero 

## model

In [59]:
print(events_df.model.unique())
print("\n\tlen: " + str(len(events_df.model.unique())))

['Samsung Galaxy J7 Prime' 'iPhone 5s' 'iPhone 6S' 'iPhone 7'
 'LG G4 H818P' nan 'iPhone 6' 'iPhone 6 Plus' 'Motorola Moto G5 Plus'
 'Motorola Moto G4 Plus' 'iPad Air Wi-Fi' 'Samsung Galaxy S6 Flat'
 'Samsung Galaxy S8 Plus' 'Samsung Galaxy S5' 'iPhone SE'
 'Samsung Galaxy S7' 'Motorola Moto X2' 'Motorola Moto G3 4G'
 'Motorola Moto X Play 4G Dual' 'Motorola Moto G4 Play'
 'Samsung Galaxy J5' 'Samsung Galaxy S5 New Edition Duos '
 'Motorola Moto G3 HDTV' 'Motorola Moto G5 ' 'Samsung Galaxy J3'
 'Samsung Galaxy S6 Edge' 'Samsung Galaxy Note 8' 'Sony Xperia Z2'
 'Motorola Moto G2 3G Dual' 'Motorola Moto Z Play' 'Samsung Galaxy S8'
 'Samsung Galaxy On 7' 'Samsung Galaxy S7 Edge' 'iPhone 8'
 'iPhone 6S Plus' 'LG K10 Novo' 'Samsung Galaxy Core Plus Duos TV'
 'iPhone 5c' 'iPhone 4S' 'Motorola Moto X Style' 'iPhone 7 Plus'
 'Samsung Galaxy A5 2016' 'Samsung Galaxy A5' 'Motorola Moto G5S Plus'
 'Samsung Galaxy J7' 'Samsung Galaxy A3 2016' 'Samsung Galaxy S3 i9300'
 'Sony Xperia M4 Aqua Dual' '

In [101]:
diccionario_modelo = generar_dict(train_df.model)
#diccionario_modelo

In [102]:
modelo_numero = generar_columna(train_df['model'], diccionario_modelo)

In [105]:
len(modelo_numero)

2341681

In [103]:
train['model_n'] = modelo_numero 

## sku

In [61]:
print(events_df.sku.unique())
print("\n\tlen: " + str(len(events_df.sku.unique())))

[ 9288.   304.  6888. ...  3851.   926. 13851.]

	len: 2329


## url

In [63]:
print(events_df.url.unique())
print("\n\tlen: " + str(len(events_df.url.unique())))

[nan '/' '/comprar/iphone/iphone-5s'
 '/comprar/samsung/galaxy-s6-edge-plus' '/comprar/samsung'
 '/comprar/samsung/' '/comprar/iphone/iphone-5' '/comprar/iphone/6s'
 '/comprar/motorola' '/comprar/samsung/galaxy-s7'
 '/comprar/iphone/iphone-5c' '/comprar/iphone/iphone-4s'
 '/comprar/samsung/galaxy-s7-edge'
 '/comprar/motorola/motorola-moto-g-3a-geracao-hdtv'
 '/comprar/motorola/moto-x-play-4g-dual' '/comprar/iphone/iphone-6s-plus'
 '/comprar/motorola/moto-g-2a-geracao' '/comprar/samsung/galaxy-s6-edge'
 '/comprar/iphone/iphone-se'
 '/comprar/motorola/motorola-moto-g-3a-geracao'
 '/comprar/samsung/galaxy-s8' '/comprar/samsung/galaxy-note-5'
 '/comprar/samsung/galaxy-a3-2016' '/comprar/iphone/iphone-5s/'
 '/comprar/samsung/galaxy-gan-2-duos-tv' '/comprar/iphone/7-plus'
 '/comprar/motorola/moto-z-play'
 '/comprar/samsung/samsung-galaxy-j2-4g-duos-tv' '/comprar/iphone/'
 '/comprar/lg/lg-k10' '/comprar/samsung/galaxy-j5'
 '/comprar/lg/lg-nexus-5-d821' '/comprar/samsung/galaxy-win-duos'
 '/co

In [104]:
diccionario_url = generar_dict(train_df.url)
url_numero = generar_columna(train_df['url'], diccionario_url)

In [172]:
len(url_numero) == events_df.shape[0]

True

In [105]:
train['url_n'] = url_numero 

## person

In [65]:
print(events_df.person.unique())
print("\n\tlen: " + str(len(events_df.person.unique())))

['4886f805' 'ad93850f' '0297fc1e' ... 'd8d6b0da' '40bf23ab' '80aea0a0']

	len: 38829


In [106]:
diccionario_person = generar_dict(train_df.person)
person_numero = generar_columna(train_df['person'], diccionario_person)

In [175]:
len(person_numero) == events_df.shape[0]

True

In [107]:
train['person_n'] = person_numero 

## event

In [67]:
print(events_df.event.unique())
print("\n\tlen: " + str(len(events_df.event.unique())))

['viewed product' 'searched products' 'ad campaign hit' 'staticpage'
 'checkout' 'search engine hit' 'conversion' 'generic listing'
 'brand listing' 'visited site' 'lead']

	len: 11


In [108]:
diccionario_event = generar_dict(train_df.event)
event_numero = generar_columna(train_df['event'], diccionario_event)

In [178]:
len(event_numero) == events_df.shape[0]

True

In [109]:
train['event_n'] = event_numero 

##Let's bring them all togheter!

In [111]:
train_df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,year,weekday,hour,year_month_day,month_sin,month_cos,day_sin,day_cos,screen_width,screen_height
0,2018-05-18 00:11:27,viewed product,ad93850f,NaN,304.0,iPhone 5s,Muito Bom,32GB,Cinza espacial,NaN,...,2018,Friday,0,2018-05-18,0.5,-0.866025,-0.485302,-0.874347,0,0
1,2018-05-18 00:23:33,viewed product,ad93850f,NaN,318.0,iPhone 5s,Muito Bom,64GB,Prateado,NaN,...,2018,Friday,0,2018-05-18,0.5,-0.866025,-0.485302,-0.874347,0,0
2,2018-05-18 00:16:10,viewed product,ad93850f,NaN,5907.0,iPhone 5s,Bom - Sem Touch ID,16GB,Cinza espacial,NaN,...,2018,Friday,0,2018-05-18,0.5,-0.866025,-0.485302,-0.874347,0,0
3,2018-05-18 00:14:55,viewed product,ad93850f,NaN,6023.0,iPhone 5s,Bom - Sem Touch ID,16GB,Dourado,NaN,...,2018,Friday,0,2018-05-18,0.5,-0.866025,-0.485302,-0.874347,0,0
4,2018-05-18 00:11:26,ad campaign hit,ad93850f,/comprar/iphone/iphone-5s,NaN,NaN,NaN,NaN,NaN,NaN,...,2018,Friday,0,2018-05-18,0.5,-0.866025,-0.485302,-0.874347,0,0


In [113]:
train['q1'] = train_df['q1']
train['q2'] = train_df['q2']
train['q3'] = train_df['q3']
train['q4'] = train_df['q4']
train['q5'] = train_df['q5']
train['q6'] = train_df['q6']
train['q7'] = train_df['q7']
train['q8'] = train_df['q8']
train['q9'] = train_df['q9']
train['q10'] = train_df['q10']
train['support'] = train_df['support_p_todos_sus_ev']
train['screen_width'] = train_df['screen_width']
train['screen_height'] = train_df['screen_height']
train['sku'] = train_df['sku']
train['year'] = train_df['year']
#train['weekday'] = train_df['weekday']
#train = train.drop(columns=['weekday'])
train['hour'] = train_df['hour']
train['month_sin'] = train_df['month_sin']
train['month_cos'] = train_df['month_cos']
train['day_sin'] = train_df['day_sin']
train['day_cos'] = train_df['day_cos']
train['label'] = train_df['label']

In [115]:
train_df.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'operating_system_version',
       'browser_version', 'mes', 'dia', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6',
       'q7', 'q8', 'q9', 'q10', 'support_p_todos_sus_ev', 'label', 'year',
       'weekday', 'hour', 'year_month_day', 'month_sin', 'month_cos',
       'day_sin', 'day_cos', 'screen_width', 'screen_height'],
      dtype='object')

In [123]:
train.to_csv('train.csv', index=False)

## Leo el train


In [16]:
#Tiro las cols no numericas
train_df.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'mes', 'dia', 'q1', 'q2',
       'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10',
       'support_p_todos_sus_ev', 'support_con_peso', 'support_ind', 'year',
       'month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin',
       'weekday_cos', 'hour_sin', 'hour_cos', 'label'],
      dtype='object')

In [43]:
train = train_df[['sku', 'mes', 'dia', 'q1', 'q2',
       'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10',
       'support_p_todos_sus_ev', 'year',
       'month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin',
       'weekday_cos', 'hour_sin', 'hour_cos', 'label']]

In [44]:
train.shape


(1171886, 24)

In [33]:
to_predict.columns

Index(['timestamp', 'event', 'person', 'url', 'sku', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version', 'mes', 'dia', 'q1', 'q2',
       'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10',
       'support_p_todos_sus_ev', 'support_con_peso', 'support_ind', 'year',
       'month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin',
       'weekday_cos', 'hour_sin', 'hour_cos'],
      dtype='object')

In [45]:
to_predict_num = to_predict[['sku', 'mes', 'dia', 'q1', 'q2',
       'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10',
       'support_p_todos_sus_ev', 'year',
       'month_sin', 'month_cos', 'day_sin', 'day_cos', 'weekday_sin',
       'weekday_cos', 'hour_sin', 'hour_cos']]

In [36]:
to_predict_num.shape

(1169795, 25)

## EMPEZAR ACA

In [2]:
import pandas as pd

In [4]:
#Leo el csv events
events_df = pd.read_csv('/home/miki_mustard/Desktop/Facultad/Datos/TP2/events_with_features.csv', low_memory=False)

In [57]:
events_df.head()

,person,sku,mes,dia,q1,q2,q3,q4,q5,q6,...,event_count_q1,event_count_q2,event_count_q3,event_count_q4,event_count_q5,event_count_q6,event_count_q7,event_count_q8,event_count_q9,event_count_q10
0,4886f805,9288.0,5,18,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
1,ad93850f,304.0,5,18,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,52.0
2,0297fc1e,6888.0,5,18,False,False,False,False,False,False,...,23.0,97.0,19.0,29.0,48.0,39.0,46.0,93.0,47.0,126.0
3,2d681dd8,11890.0,5,18,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0
4,cccea85e,7517.0,5,18,False,False,False,False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,387.0,449.0


In [28]:
events_df = events_df.drop(columns=['timestamp', 'event', 'url', 'model', 'condition',
       'storage', 'color', 'skus', 'search_term', 'staticpage',
       'campaign_source', 'search_engine', 'channel', 'new_vs_returning',
       'city', 'region', 'country', 'device_type', 'screen_resolution',
       'operating_system_version', 'browser_version'])

KeyError: "['timestamp' 'event' 'url' 'model' 'condition' 'storage' 'color' 'skus'\n 'search_term' 'staticpage' 'campaign_source' 'search_engine' 'channel'\n 'new_vs_returning' 'city' 'region' 'country' 'device_type'\n 'screen_resolution' 'operating_system_version' 'browser_version'] not found in axis"

In [29]:
events_df = events_df.drop(columns=['tiempo_total_q4', 'tiempo_total_q5', 'tiempo_total_q6',
       'tiempo_total_q7', 'tiempo_total_q8', 'tiempo_total_q9',
       'tiempo_total_q10','tiempo_total'])

In [36]:
events_df = events_df.drop(columns=['tiempo_total_q1', 'tiempo_total_q2', 'tiempo_total_q3'])

In [11]:
#Leo el csv labels
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [37]:
#Creo el dataframe con las personas que tienen labels
# armo df con registros completos clasificados
train_df = events_df.merge(labels_df.set_index("person"), on='person', how='right')

In [38]:
# armo df con registros a predecir unicamente
to_predict = events_df[~events_df.person.isin(labels_df.person)]

In [54]:
to_predict_num = to_predict.drop(columns=['person'])

In [79]:
to_predict_num.columns

Index(['sku', 'mes', 'dia', 'q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8',
       'q9', 'q10', 'event_count', 'event_count_q1', 'event_count_q2',
       'event_count_q3', 'event_count_q4', 'event_count_q5', 'event_count_q6',
       'event_count_q7', 'event_count_q8', 'event_count_q9',
       'event_count_q10'],
      dtype='object')

In [39]:
train = train_df.drop(columns=['person'])

In [14]:
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression

In [40]:
X, y = train.iloc[:,:-1],train.iloc[:,-1]

In [41]:
X.dtypes

sku                float64
mes                  int64
dia                  int64
q1                    bool
q2                    bool
q3                    bool
q4                    bool
q5                    bool
q6                    bool
q7                    bool
q8                    bool
q9                    bool
q10                   bool
event_count          int64
event_count_q1     float64
event_count_q2     float64
event_count_q3     float64
event_count_q4     float64
event_count_q5     float64
event_count_q6     float64
event_count_q7     float64
event_count_q8     float64
event_count_q9     float64
event_count_q10    float64
dtype: object

In [124]:
y.value_counts()

0    1072202
1      99684
Name: label, dtype: int64

In [20]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [42]:
data_dmatrix = xgb.DMatrix(data=X,label=y)

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.3, random_state=100)

In [44]:
len(y_train)

820320

In [45]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', booster='dart',
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 20, n_estimators = 80)

In [46]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=20, base_score=0.5, booster='dart', colsample_bylevel=1,
       colsample_bytree=0.3, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [47]:
preds = xg_reg.predict(X_test)

In [48]:
preds

array([2.2992224e-02, 5.4069650e-01, 1.4373690e-02, ..., 3.0109286e-04,
       1.2776345e-02, 1.3920158e-02], dtype=float32)

In [51]:
roc_auc_score(y_test,preds)

0.9696089819581609

In [52]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.146366


In [32]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,
          'learning_rate': 0.1, 'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="auc", as_pandas=True, seed=123)

[11:53:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[11:53:30] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[11:53:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[11:53:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[11:53:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=5
[11:53:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[11:53:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=4
[11:53:32] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[11:53:33] /workspace/

In [55]:
preds_posta = xg_reg.predict(to_predict_num)

In [56]:
preds_posta

array([0.0189572 , 0.10821775, 0.02357987, ..., 0.05523124, 0.04629332,
       0.09116453], dtype=float32)

In [59]:
to_predict.shape

(1169795, 47)

In [100]:
# armo df con registros a predecir unicamente
#to_predict = event[~event.person.isin(label.person)]

In [60]:
to_publish = pd.DataFrame()

In [61]:
to_publish['person'] = to_predict.person

In [62]:
to_publish['label'] = preds_posta

In [63]:
to_publish.groupby('person', as_index=False).mean().to_csv('miki_xgb1.csv', index=False)

In [65]:
to_publish.head()

,person,label
0,4886f805,0.030412
2,0297fc1e,0.105346
3,2d681dd8,0.049659
4,cccea85e,0.061076
6,29ebb414,0.035675


In [90]:
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

RMSE: 0.269414


In [91]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,
          'learning_rate': 0.1, 'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=50, early_stopping_rounds=10,
                    metrics="rmse", as_pandas=True, seed=123)

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7fdfa392fc18>>
Traceback (most recent call last):
  File "/home/miki_mustard/envdatos/lib/python3.6/site-packages/xgboost/core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


[18:15:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[18:15:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[18:15:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 22 extra nodes, 0 pruned nodes, max_depth=5
[18:15:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 26 extra nodes, 0 pruned nodes, max_depth=5
[18:15:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18 extra nodes, 0 pruned nodes, max_depth=5
[18:15:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pruned nodes, max_depth=1
[18:15:59] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 32 extra nodes, 0 pruned nodes, max_depth=5
[18:16:00] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=5
[18:16:00] /works

In [146]:
cv_results.head()

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.553673,0.000521,0.553632,0.000691
1,0.642067,0.037459,0.642438,0.036788
2,0.678191,0.039707,0.678782,0.040165
3,0.713285,0.017182,0.713565,0.017668
4,0.715148,0.020791,0.715348,0.021529


In [148]:
print((cv_results["test-auc-mean"]).tail(1))

49    0.821954
Name: test-auc-mean, dtype: float64
